In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [75]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


In [76]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [77]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [78]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [192]:
#preparing data for splitting

data_new = pd.read_csv('Crash_Table.csv')
row, col = data_new.shape

for index, row in data_new.iterrows():
    if row["egoinvolve"] == "No":
        data_new = data_new.drop(index)
data_new = data_new.drop("startframe", axis='columns')
data_new = data_new.drop("youtubeID", axis='columns')
data_new = data_new.drop("timing", axis='columns')
data_new = data_new.drop("weather", axis='columns')
data_new = data_new.drop("egoinvolve", axis='columns')

names = os.listdir("./normalframes")
header = list(data_new.columns)
data_normal = pd.DataFrame(columns=header)

for i in names:
    data_normal.loc[len(data_normal.index)] = [i, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
data_new = pd.concat([data_normal, data_new], axis=0)
print(data_new)
data_new.drop(data_new.iloc[:, 1:50], inplace=True, axis=1)
print(data_new)


     vidname  frame_1  frame_2  frame_3  frame_4  frame_5  frame_6  frame_7  \
0     000787        0        0        0        0        0        0        0   
1     001655        0        0        0        0        0        0        0   
2     001467        0        0        0        0        0        0        0   
3     002686        0        0        0        0        0        0        0   
4     001003        0        0        0        0        0        0        0   
...      ...      ...      ...      ...      ...      ...      ...      ...   
1494    1495        0        0        0        0        0        0        0   
1495    1496        0        0        0        0        0        0        0   
1496    1497        0        0        0        0        0        0        0   
1497    1498        0        0        0        0        0        0        0   
1499    1500        0        0        0        0        0        0        0   

      frame_8  frame_9  ...  frame_41  frame_42  fr

In [267]:
'''train_df = data_new.sample(frac=0.75).astype(str)
test_df = data_new.drop(train_df.index).astype(str)'''


# Using DataFrame.sample() 
train_df=data_new.sample(frac=0.8,random_state=200)
test_df=data_new.drop(train_df.index)

print(train_df)
print(test_df)

     vidname  frame_50
105      106         1
1138    1139         1
1841  001465         0
1838  000543         0
2649  000798         0
...      ...       ...
2086  002960         0
639      640         1
2482  001930         0
80    000745         0
1793  002473         0

[3042 rows x 2 columns]
     vidname  frame_50
18    001458         0
73    002482         0
76    001451         0
84    002644         0
89    002449         0
...      ...       ...
1262    1263         1
1272    1273         1
1326    1327         1
1399    1400         1
1485    1486         1

[537 rows x 2 columns]


In [268]:
import shutil
import glob

path = "/Users/nalthan/Desktop/vertiasopencvprojecy/"
normal_file_names = [path_normal + "/" + filename for filename in os.listdir(path_normal) if filename.endswith('.mp4')]

'''for ind in train_df.index:
    from IPython.core.debugger import Pdb; Pdb().set_trace()
    if train_df.loc[ind, "frame_50"].where() == 1:
        name = train_df.loc[ind, "vidname"]
        length = 6 -len(name)
        name = name.zfill(length)
        print(name)
        os.rename(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
    if train_df.loc[ind, "frame_50"].item() == 0:
        name = train_df.loc[ind, "vidname"]
        print(name)
        os.rename(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")'''
'''for f in glob.glob(path+"train"):
    print(f)
    os.remove(f)
for f in glob.glob(path+"test"):
    print(f)
    os.remove(f)'''

for row in train_df.iterrows():
    if str(row[1][0]) == ".DS_Store":
        continue
    if row[1][1] == 1:
        name = str(row[1][0])
        name = name.zfill(6)
        #os.rename(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
        shutil.copy(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
    else:
        name = row[1][0]
        #os.rename(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")
        shutil.copy(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")
        
for row in test_df.iterrows():
    if row[1][1] == 1:
        name = str(row[1][0])
        name = name.zfill(6)
        #os.rename(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
        shutil.copy(path+"Crash-1500/"+name+".mp4", path+"test/"+name+".mp4")
    else:
        name = row[1][0]
        #os.rename(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")
        shutil.copy(path+"Normal/"+name+".mp4", path+"test/"+name+".mp4")

In [269]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [270]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


In [2]:
print(train_df)

'''label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df.iloc[:, 1])
)
print(label_processor.get_vocabulary())'''


NameError: name 'train_df' is not defined

In [321]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["vidname"].values.tolist()
    labels = df["frame_50"].values

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(idx, path)
        print(type(path))
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, str(path).zfill(6)+'.mp4'))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "/Users/nalthan/Desktop/vertiasopencvprojecy/train")
test_data, test_labels = prepare_all_videos(test_df, "/Users/nalthan/Desktop/vertiasopencvprojecy/test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")


0 106
<class 'int'>


[ WARN:0@34800.447] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@34800.447] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3585 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@34800.447] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@34800.447] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 36ms/step
1 1139
<class 'int'>


[ WARN:0@34802.086] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@34802.086] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3586 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@34802.086] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@34802.086] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 33ms/step
2 001465
<class 'str'>


[ WARN:0@34803.450] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@34803.450] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3587 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@34803.450] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@34803.450] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 36ms/step
3 000543
<class 'str'>


[ WARN:0@34804.699] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@34804.699] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3588 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@34804.699] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@34804.699] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 33ms/step


KeyboardInterrupt: 

In [1]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

NameError: name 'keras' is not defined

In [354]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["vidname"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction("/Users/nalthan/Desktop/vertiasopencvprojecy/test/001486.mp4")
#to_gif(test_frames[:MAX_SEQ_LENGTH])


Test video path: 001618


[ WARN:0@36310.621] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@36310.621] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3602 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@36310.621] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@36310.621] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 18ms/step
  0: 53.34%
  1: 46.66%


[[[0.55690026 0.00372693 1.164155   ... 0.5683828  0.2273449  0.27284822]
  [0.5518104  0.         0.56378764 ... 0.32555878 0.20508468 0.37070766]
  [0.7172964  0.         0.57053643 ... 0.4331803  0.16984011 0.34802505]
  ...
  [0.5535542  0.12226056 0.30052516 ... 0.8323538  0.24497993 0.02150873]
  [0.7964966  0.258994   0.20246062 ... 0.9666561  0.43305612 0.15012342]
  [0.42056656 0.23802823 0.12534954 ... 1.0736641  0.2836176  0.06860691]]

 [[0.9625615  0.13510786 0.6706079  ... 0.21455297 0.21543796 0.9329506 ]
  [0.86544126 0.09389149 0.6420261  ... 0.46122724 0.10112777 1.0568451 ]
  [1.147747   0.1785585  0.69596744 ... 0.47264764 0.05745816 0.9192556 ]
  ...
  [0.8775185  0.2427949  0.5206336  ... 0.18757854 0.17429346 0.60987484]
  [0.7811601  0.14033282 0.5562525  ... 0.27848884 0.5472677  0.54624724]
  [0.77163225 0.13750356 0.57990557 ... 0.30191937 0.66745603 0.7738948 ]]

 [[0.5100756  0.17153814 0.71176195 ... 0.39772078 0.4923519  0.6491169 ]
  [0.23585825 0.095093